In [1]:
%matplotlib inline
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage import io
from skimage import morphology
import operator
import pickle
from skimage.color import label2rgb
from skimage.measure import regionprops
import os

In [2]:
def lista_imagens(path, blacklist, max_items=5):
    addr = []
    folder = os.listdir(path)

    for i in folder:
        if i not in blacklist:
            addr.append(path + "/" + i)
        else:
            pass
    return addr

In [3]:
blacklist = []

In [4]:
image_list = lista_imagens("C:/Users/jorge/Dropbox/BancoDeImagens/Sudoku", blacklist)

In [5]:
image_list

['C:/Users/jorge/Dropbox/BancoDeImagens/Sudoku/arthurqmo-0.png',
 'C:/Users/jorge/Dropbox/BancoDeImagens/Sudoku/arthurqmo-0.txt',
 'C:/Users/jorge/Dropbox/BancoDeImagens/Sudoku/arthurqmo-1.png',
 'C:/Users/jorge/Dropbox/BancoDeImagens/Sudoku/arthurqmo-1.txt',
 'C:/Users/jorge/Dropbox/BancoDeImagens/Sudoku/arthurqmo-2.png',
 'C:/Users/jorge/Dropbox/BancoDeImagens/Sudoku/arthurqmo-2.txt',
 'C:/Users/jorge/Dropbox/BancoDeImagens/Sudoku/arthurqmo-3.png',
 'C:/Users/jorge/Dropbox/BancoDeImagens/Sudoku/arthurqmo-3.txt',
 'C:/Users/jorge/Dropbox/BancoDeImagens/Sudoku/arthurqmo-4.png',
 'C:/Users/jorge/Dropbox/BancoDeImagens/Sudoku/arthurqmo-4.txt',
 'C:/Users/jorge/Dropbox/BancoDeImagens/Sudoku/brunoacpc-1.png',
 'C:/Users/jorge/Dropbox/BancoDeImagens/Sudoku/brunoacpc-1.txt',
 'C:/Users/jorge/Dropbox/BancoDeImagens/Sudoku/brunoacpc-2.png',
 'C:/Users/jorge/Dropbox/BancoDeImagens/Sudoku/brunoacpc-2.txt',
 'C:/Users/jorge/Dropbox/BancoDeImagens/Sudoku/brunoacpc-3.png',
 'C:/Users/jorge/Dropbox/

In [7]:
tile_gabarito = []

for imagem in range(0, len(image_list), 2):
    print(image_list[imagem])
    img_sudoku = cv2.cvtColor(cv2.imread(image_list[imagem]), cv2.COLOR_BGR2GRAY) # carrega a imagem escolhida
    sudoku_test = cv2.adaptiveThreshold(img_sudoku, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 15, 5) #transformar a imagem em binária com threshold adaptativo
    w, h = sudoku_test.shape

    contours, hierarchy = cv2.findContours(sudoku_test, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    if len(contours) != 0:
        c2 = max(contours, key = cv2.contourArea)

    corners = []
    i = 0
    flag1 = False
    while len(corners) != 4:
        corners = cv2.approxPolyDP(c2, i, True)
        i += 1
        if i > 300:
            flag1 = True
            break
    
    if flag1:
        continue

    for corner in corners:
        if corner[0][0] < w/2:
            if corner[0][1] < h/2:
                ul = corner[0] # upper left
            else:
                ll = corner[0] # lower left
        else:
            if corner[0][1] < h/2:
                ur = corner[0] # upper right
            else:
                lr = corner[0] # lower right

    cols, rows = 2000, 2000 # redimensionar para o plot não ficar achatado

    original = np.float32([ul, ll, ur, lr])
    perspec = np.float32([[0, 0], [0, rows], [cols, 0], [cols, rows]])


    M = cv2.getPerspectiveTransform(original, perspec)
    grid_crop = cv2.warpPerspective(sudoku_test, M, (cols, rows))

    grid_crop = cv2.rectangle(grid_crop, (1,1), (cols-1, rows-1), (255,255,255), thickness=30)

    grid_crop = morphology.remove_small_objects(grid_crop, min_size=9000)
    
    img = morphology.dilation(grid_crop, morphology.square(5)) > 0 # dilation para deixas as linhas mais largas, para que os quadrados (props) fiquem bem definidos
    img1 = morphology.remove_small_objects(img, min_size=100)
    img1 = morphology.remove_small_holes(img1, 110, connectivity=2)
    img1 = morphology.remove_small_holes(img1, 110)
    
    verticals = morphology.erosion(img1, morphology.rectangle(130, 1))
    img2 = morphology.reconstruction(verticals, img1)
    
    img3 = morphology.dilation(img2, morphology.square(5)) > 0

    img = ~img3
    
    props = regionprops(morphology.label(img))

#     fig, axs = plt.subplots(9,9, figsize=(7, 7)) # setup do plot para plotar
#     axs = axs.ravel()

    prop_list = []
    crop_list = []

    soma = 0
    for i in props:
        soma += i.area

    soma /= len(props)

    props = [i for i in props if i.area > 0.50 * soma] # selecionar os props desejados (aqueles com área desejada).

    
    if len(props) != 81:
        continue
    
#     print(len(props))

    for i in range(81):
        prop_list.append(props[i].bbox)

    chunks = [prop_list[x:x+9] for x in range(0, len(prop_list), 9)] # percorrer a lista de props de 9 em 9, adicionando cada linha do sudoku em uma lista separada
    for i in chunks:
        i.sort(key = operator.itemgetter(1)) # ordenação da lista pelo segundo elemento (posição da primeira coluna do prop), pois os props vêm ordenados em linhas, mas a ordem das linhas pode estar errada.
    chunks = [item for sublist in chunks for item in sublist] # reagrupar as listas de props em uma só

    tem_ou_nao = []
    tiles = []
    hist_list = []

#     a = 0
    for i in chunks: # criar os respectivos crops de cada prop, já plotando cada um em sua respectiva posição
        crop = grid_crop[i[0]:i[2], i[1]:i[3]]
        tiles.append(crop)
        soma = np.sum(grid_crop[i[0]:i[2], i[1]:i[3]])
        tem_ou_nao.append(soma/((i[2]-i[0]) * (i[3]-i[1])))

#         axs[a].contourf(crop[::-1])
#         a+=1
    
    media = np.sum(tem_ou_nao) / len(tem_ou_nao)

    # print(media)

    tem_ou_nao_bool = tem_ou_nao > media # filtrar os tiles que têm números para impedir predicts desnecessários e errados.
    tem_ou_nao_bool = tem_ou_nao_bool.reshape((9,9))
    tiles = np.array(tiles).reshape((9,9))
    
    gabarito = open(image_list[imagem+1], 'r').readlines()

    for i in range(len(gabarito)):
        gabarito[i] = gabarito[i].replace('\n', "")
    
    for i in range(9):
        for j in range(9):
            if tem_ou_nao_bool[i][j]:
                if gabarito[i][j] == '-':
                    print('benis')
                    continue
                else:
                    tile_gabarito.append((cv2.resize(tiles[i][j], (28, 28)), int(gabarito[i][j])))

C:/Users/jorge/Dropbox/BancoDeImagens/Sudoku/arthurqmo-0.png
C:/Users/jorge/Dropbox/BancoDeImagens/Sudoku/arthurqmo-1.png
C:/Users/jorge/Dropbox/BancoDeImagens/Sudoku/arthurqmo-2.png
C:/Users/jorge/Dropbox/BancoDeImagens/Sudoku/arthurqmo-3.png
C:/Users/jorge/Dropbox/BancoDeImagens/Sudoku/arthurqmo-4.png
C:/Users/jorge/Dropbox/BancoDeImagens/Sudoku/brunoacpc-1.png
C:/Users/jorge/Dropbox/BancoDeImagens/Sudoku/brunoacpc-2.png
C:/Users/jorge/Dropbox/BancoDeImagens/Sudoku/brunoacpc-3.png
C:/Users/jorge/Dropbox/BancoDeImagens/Sudoku/brunoacpc-4.png
C:/Users/jorge/Dropbox/BancoDeImagens/Sudoku/brunoacpc-5.png
C:/Users/jorge/Dropbox/BancoDeImagens/Sudoku/davidf2-0.png
C:/Users/jorge/Dropbox/BancoDeImagens/Sudoku/davidf2-1.png
C:/Users/jorge/Dropbox/BancoDeImagens/Sudoku/davidf2-2.png
C:/Users/jorge/Dropbox/BancoDeImagens/Sudoku/davidf2-3.png
C:/Users/jorge/Dropbox/BancoDeImagens/Sudoku/davidf2-4.png
C:/Users/jorge/Dropbox/BancoDeImagens/Sudoku/davidf2-5.png
C:/Users/jorge/Dropbox/BancoDeImagen

In [8]:
len(tile_gabarito)

757

In [10]:
file = open('gabarito', 'wb')
pickle.dump(tile_gabarito, file)
file.close()